# ISH PHANDA
# 102016098
# 3CS12

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

def calculate_perplexity(generated_text):
    input_ids = tokenizer.encode(generated_text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits
    loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1), reduction="none")
    perplexity = torch.exp(torch.mean(loss))
    return perplexity.item()

def calculate_fluency_score(generated_text):
    perplexity = calculate_perplexity(generated_text)
    return 1 / perplexity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def calculate_diversity_score(generated_text):
    words = generated_text.split()
    unique_words = len(set(words))
    total_words = len(words)
    diversity_score = unique_words / total_words
    return diversity_score

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
import nltk
from nltk.corpus import brown

brown_bigrams = set(nltk.bigrams(brown.words()))

def calculate_coherence_score(generated_text):
    words = generated_text.split()
    bigrams = list(zip(words[:-1], words[1:]))
    common_bigrams = sum(bigram in brown_bigrams for bigram in bigrams)
    coherence_score = common_bigrams / len(bigrams)
    return coherence_score

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
from transformers import pipeline

models = [('distilgpt2', 'distilgpt2-small'),('gpt2', 'gpt2-small'),('gpt2-medium', 'gpt2-medium')]

criteria = {'fluency': 0.3, 'diversity': 0.3, 'coherence': 0.2}

def generate_text_and_calculate_scores(model_name, model_type):
    generator = pipeline('text-generation', model=model_name)
    generated_text = generator("Once upon a time", max_length=30, do_sample=False)
    print("Generated text for model", model_name, ":", generated_text[0]['generated_text'])
    generated_text = generated_text[0]['generated_text']
    fluency_score = calculate_fluency_score(generated_text)
    diversity_score = calculate_diversity_score(generated_text)
    coherence_score = calculate_coherence_score(generated_text)
    return (model_name, model_type, fluency_score, diversity_score, coherence_score)

data = []
for model_name, model_type in models:
    scores = generate_text_and_calculate_scores(model_name, model_type)
    data.append(scores)

data = np.array(data)[:, 2:].astype(float)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text for model distilgpt2 : Once upon a time of war, the United States was the only country in the world to have a military presence. The United States was the only country


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text for model gpt2 : Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text for model gpt2-medium : Once upon a time, there was a man who lived in a village called Krakow. He was a very good man, and he was very


In [ ]:
normalized_data = data / np.sqrt((data ** 2).sum(axis=0))
weighted_data = normalized_data * np.array(list(criteria.values()))
ideal_solution = np.max(weighted_data, axis=0)
anti_ideal_solution = np.min(weighted_data, axis=0)
distance_from_ideal = np.sqrt(((weighted_data - ideal_solution) ** 2).sum(axis=1))
distance_from_anti_ideal = np.sqrt(((weighted_data - anti_ideal_solution) ** 2).sum(axis=1))
topsis_score = distance_from_anti_ideal / (distance_from_ideal + distance_from_anti_ideal)
ranked_models = np.argsort(topsis_score)
print("Ranked Models (from best to worst):")
for rank, model_idx in enumerate(ranked_models):
    model_name, model_type = models[model_idx]
    print(f"Rank {rank + 1}: Model {model_name} ({model_type}) with TOPSIS score {topsis_score[model_idx]:.2f}")

Ranked Models (from best to worst):
Rank 1: Model gpt2 (gpt2-small) with TOPSIS score 0.13
Rank 2: Model distilgpt2 (distilgpt2-small) with TOPSIS score 0.41
Rank 3: Model gpt2-medium (gpt2-medium) with TOPSIS score 0.86
